In [29]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth', 'Utils', 'Other', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = '10 Industry Portfolios - Average Value Weighted Returns.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [31]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    rf = pd.read_csv(file_name, delimiter = ',', header = 0, usecols = ['RF'])
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = data

actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
date,,,,,,,,,,,
196307,-0.47,-0.22,-1.41,2.29,-0.69,-0.23,-1.03,0.56,0.80,-1.61,-0.12
196308,4.87,6.55,6.20,3.93,5.14,4.29,6.40,9.56,4.20,5.49,5.32
196309,-1.68,-0.24,-0.76,-3.64,0.13,2.36,0.96,-4.06,-2.50,-3.16,-1.30
196310,2.66,9.72,2.58,-0.32,8.29,3.40,0.48,3.38,-0.67,1.38,2.82
196311,-1.13,-4.84,0.30,-1.15,-0.29,4.16,-1.23,-1.65,-1.02,0.23,-0.58


In [32]:
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

In [33]:
new_returns_data = returns_data
returns_monthly = new_returns_data
returns_mean = returns_monthly.mean()
cov_matrix = returns_monthly.cov()
    
# udregner profit, volatilitet og sharpe
tl = np.dot(np.linalg.inv(cov_matrix), np.ones(11))
nl = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), np.ones(11)))
weights = tl / nl
    
# udregner afkast for måned T + 1 med de udregnede optimale vægte
returns = np.dot(weights, returns_data.mean()) - float(rf.mean())
volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
sharpe = returns / volatility

# indsætter overstående udregninger i vektorerne
sharpe_ratio.append(sharpe)
port_returns.append(returns)
port_volatility.append(volatility)
stock_weights.append(weights)

In [34]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

In [35]:
# definerer de enkelte vægte af de valgte aktiver, HINT: de er 0.33 alle sammen da det er 1/N
for i in range(len((actives))):
    symbol = actives[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [36]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in actives]
df = df[column_order]

df.head() # Porteføljer ud fra 60 måneders interval
          # ie. portfølje '0' har return, vol og sharpe ud fra måned 1 - 60
          # portfølje '1' har således overstående ud fra måned 2 - 61 ovs.

,Returns,Volatility,Sharpe Ratio,NoDur Weight,Durbl Weight,Manuf Weight,Enrgy Weight,HiTec Weight,Telcm Weight,Shops Weight,Hlth Weight,Utils Weight,Other Weight,Mkt Weight
0,0.443232,3.421246,0.129553,0.244188,0.060185,0.230162,0.156857,0.025954,0.290389,-0.018999,0.132189,0.453433,-0.517106,-0.057252


In [37]:
df['Returns'].mean() / df['Returns'].std()

nan